In [4]:
shortagecalculator.pred_data[shortagecalculator.pred_data['sku']=='340001-RGB-US-2'

,sku,country,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,pred_2021-05,pred_2021-06,pred_2021-07,pred_2021-08,pred_2021-09,pred_2021-10
163,340001-RGB-US-2,CA,300,400,600,500,500,600,300,700,1300,1800,2300,2900
164,340001-RGB-US-2,US,280,380,300,320,300,600,280,660,960,1280,1580,2180


In [6]:
import pandas as pd
sales_data['purchase_date'] = pd.to_datetime(sales_data['purchase_date'])

In [11]:
import numpy as np
sales_data = sales_data.groupby(['sku','purchase_date'])['quantity'].agg(np.sum).reset_index()

In [ ]:
from shortagecalculator import shortagecalculator
shortagecalculator.get_sto_data()

In [1]:
from battery import Battery
battery = Battery(data_module='sales',country='us')
battery.model.country
battery = Battery(data_module='sales',country='gb')
battery.get_data(data='sales_data')
sales = Battery(data_module='sales',country='gb')

In [1]:
class Subscriber:
    def __init__(self, name):
        self.name = name
        self.history = {}

    def update(self, event, message):
        print(f'{self.name} got message: {message}')
        self.history.update( {self.name:{event: message}} )

class Publisher:
    def __init__(self, events):
        # self.name = name
        self.subscribers = {event: dict() for event in events}
    
    def get_subscribers(self, event):
        return self.subscribers[event]

    def register(self, event, who, callback=None):
        if not callback:
            callback = getattr(who, 'update') 
        self.get_subscribers(event)[who] = callback

    def unregister(self, event, who):
        del self.get_subscribers(event)[who]
    
    def notify(self, event, message):
        for subscriber, callback in self.get_subscribers(event).items():
            callback(event, message)

In [8]:
sales = Publisher(['sql_update','data_update'])
storage = Publisher(['sql_update','data_update'])

In [9]:
battery = Subscriber('battery')
sales.register("sql_update", battery)
storage.register("sql_update", battery)
storage.register("data_update", battery)

In [4]:
sales.notify('sql_update','update sql with parameter: country, limit')
storage.notify("data_update", 'a new type of data has been added')

battery got message: update sql with parameter: country, limit
battery got message: a new type of data has been added


In [1]:
import os
import sys

import pandas as pd
import numpy as np
import datetime
import importlib

from typing import Optional, Dict, Any
from sqlalchemy import create_engine
# from sqlalchemy.ext.asyncio import create_async_engine
from dotenv import load_dotenv
from pandas.core.frame import DataFrame
from pandas.core.series import Series

sys.path.insert(0, 'module')


class Subscriber:
    def __init__(self, name):
        self.name = name
        self.history = {}

    def update(self, data_module, event, message):
        # print(f'{self.name} received message from {data_module}: {**kwargs}')
        self.history.update({data_module: {event: message}})


class Battery(Subscriber):
    def __init__(self, data_module: str, **kwargs: Dict[str, Any]):
        super(Battery, self).__init__(name='battery')
        self.module = importlib.import_module(data_module)
        self.model_generator = getattr(self.module, data_module.capitalize())
        self.model = self.model_generator(**kwargs)

    def get_data(self, data: str):
        target_data = getattr(self.model, data)
        return target_data

    def read_sto_data(self):
        raw_sto = pd.read_excel('1,仓库每日进销存报表（无公式版）.xlsx', sheet_name=None)
        return raw_sto

    def read_pred_data(self):
        raw_pred = pd.read_excel('0430预估.xlsm', sheet_name='分站点预估')
        return raw_pred


In [31]:
import pandas as pd
from core.db import db
from typing import Dict, Any, Optional
from pandas.core.frame import DataFrame
from pandas.core.series import Series
from core.battery import Battery 


class Publisher:
    def __init__(self, events):
        self.subscribers = {event: dict() for event in events}
    
    def get_subscribers(self, event):
        return self.subscribers[event]

    def register(self, data_module, event, who, callback=None):
        if not callback:
            callback = getattr(who, 'update') 
        self.get_subscribers(event)[who] = callback

    def unregister(self, data_module, event, who):
        del self.get_subscribers(event)[who]
    
    def notify(self, data_module, event, message):
        for subscriber, callback in self.get_subscribers(event).items():
            callback(data_module, event, message)
            # print('event:', event,'message:', **kwargs)

class Storage(Publisher):
    def __init__(self, events=['sql_update','data_update'],**kwargs: Dict[str, Any]):
        super(Storage, self).__init__(events = events)
        self.__dict__.update(kwargs)
        self.sto_data = pd.DataFrame()
        self.battery = Battery(data_module=self.__class__.__name__.lower(), name='battery')
        self.register(data_module=self.__class__.__name__.lower(), event = 'sql_update', who=self.battery)
        self.notify(data_module=self.__class__.__name__.lower(), event = 'sql_update', message='changes happend')

    @property
    def storage_data(self):
        raw_sto = pd.read_excel('1,仓库每日进销存报表（无公式版）.xlsx', sheet_name=None)
        if (type(self.sto_data) != dict) and ('仓库' in self.sto_data.columns):
            return self.sto_data
        for key in list(raw_sto.keys())[1:-1]:
            raw_sto[key].rename(columns=raw_sto[key].iloc[0], inplace=True)
            raw_sto[key] = raw_sto[key].iloc[1:, :]
            raw_sto[key] = raw_sto[key][[
                'SKUNo.', 'P_SKU', '仓库', '已备未发', '海外库存']]
            raw_sto[key].rename(
                columns={'SKUNo.': 'ssku', 'P_SKU': 'p_sku'}, inplace=True)
            raw_sto[key] = raw_sto[key][~(raw_sto[key]['仓库'] == '汇总')]
            raw_sto[key]['storage'] = raw_sto[key]['已备未发'] + \
                raw_sto[key]['海外库存']   # 如果是下单则再+SZ仓库存数量(除去已经备货) + 工厂已下单的量 + 暂收仓待检/不良
            self.sto_data = self.sto_data.append(raw_sto[key])
            self.sto_data[['ssku', 'p_sku']] = self.sto_data[['ssku', 'p_sku']].applymap(
                lambda x: str(x))
        return self.sto_data

In [1]:
battery = Battery(data_module='sales',name='battery')
storage = Storage()

NameError: name 'Battery' is not defined

In [33]:
storage.register('Storage', 'sql_update', battery)
storage.notify('Storage',"sql_update", {'country':3})

battery received message from Storage: {'country': 3}
battery received message from Storage: {'country': 3}


In [1]:
from core.battery import Battery

In [2]:
battery = Battery('storage')

manager received message from storage: changes happend
event: sql_update message: changes happend


In [3]:
from datetime import datetime
import pandas as pd
import numpy as np
from core.battery import Battery

sales = Battery('sales',country='us').get_data('sales_data')
sales.groupby(['sku','date'])['sales'].agg(np.sum)
month_ago = datetime.today().date()-pd.DateOffset(months=1)
sales[sales['date']>month_ago].groupby(['sku'])['sales'].agg(np.sum)
sales

manager received message from sales: changes happend
event: sql_update message: changes happend


In [4]:
storage_data = Battery('storage').get_data('storage_data')

manager received message from storage: changes happend
event: sql_update message: changes happend


In [24]:
from core.battery import Battery
from core.manager import Manager


manager received message from prediction: changes happend
event: sql_update message: changes happend


In [6]:
from calculator.shortagecalculator import shortagecalculator

In [1]:
from core.battery import Battery
from core.manager import manager

In [2]:
predication = Battery('prediction',month=6).get_data('prediction_data')

manager received message from module prediction: instantiated at 2021-05-18 13:26:08.710578
manager received message from module prediction: retriving prediction_data


In [1]:
import inspect

class Test:
    def __init__(self):
        pass
    def foo(self,a:int=1,b:str='b',c:list=['1']):
        print(inspect.stack()[0][3])
        print(inspect.stack()[1][3])
        return inspect.getfullargspec(test.foo)
test = Test()
arg_annotations = inspect.getfullargspec(test.foo)[-1]
arg_annotations

In [1]:
from calculator.shortagecalculator import shortagecalculator

In [6]:
sto_data = shortagecalculator.get_sto_data()

manager received message from module storage: changes happend


In [8]:
sto_data[sto_data['ssku']=='3300002']

,ssku,p_sku,sto_name,stockup,oversea_storage,sz_storage,in_factory,qc,period,continue,level,storage_to_send,storage_sum
1,3300002,3300002,US01,0,25990,147,64519,4213,89,1,A,25990,94869
2,3300002,3300002,US02,0,0,147,64519,4213,85,1,A,0,68879
3,3300002,3300002,UK02,0,755,147,64519,4213,91,1,A,755,69634
4,3300002,3300002,DE01,540,13856,147,64519,4213,90,1,A,14396,83275
5,3300002,3300002,UK03,0,8014,147,64519,4213,89,1,A,8014,76893
6,3300002,3300002,DE02,0,98,147,64519,4213,88,1,A,98,68977
7,3300002,3300002,CA01,0,517,147,64519,4213,91,1,A,517,69396
8,3300002,3300002,AU01,0,0,147,64519,4213,71,0,dis,0,68879
9,3300002,3300002,JP01,0,4576,147,64519,4213,87,1,A,4576,73455


In [15]:
shortage = shortagecalculator.get_shortage()

fail copying sku


In [228]:
import pandas as pd
import numpy as np

data = pd.read_excel('us-0518.xls', sheet_name =None)

In [229]:
data.keys()

dict_keys(['Sheet1', 'Sheet1(2)'])

In [247]:
data_[data_['market_cap'].isna()][['sku','asin']].drop_duplicates('sku').reset_index().to_excel('missing_market_cap_sku.xlsx',index=False)

In [113]:
temp = data.groupby(['sku','asin','country'])['price'].agg(np.average).reset_index()
data[data['price']==0]
data[data['price']==0][['sku','date','units_ordered']].sort_values(by='date',ascending=False)
temp[temp['price']==0].to_excel('skumissingprice.xlsx')
data[data['price']==0].groupby(['sku','date'])['units_ordered'].agg(np.sum).reset_index().sort_values(by='units_ordered',ascending=False).head(50)

,sku,asin,date,country,category,market_cap,rating,rating_number,price,ads_cpc,...,asin_of_c1,rating_of_c1,rating_number_of_c1,price_of_c1,date_first_available_of_c1,asin_of_c2,rating_of_c2,rating_number_of_c2,price_of_c2,date_first_available_of_c2
1654,PR410102-RGB-WP-US,B08BZGRJCV,2020-09-09,us,led strip lights,us-th1001-2000,0.0,0.0,0.0,0.0000,...,B07JP5375R,0.0,0.0,0.0,2018-10-25,B07RY76LGX,0.0,0.0,0.0,2019-05-17
1655,PR410102-RGB-WP-US,B08BZGRJCV,2020-09-10,us,led strip lights,us-th1001-2000,0.0,0.0,0.0,0.7500,...,B07JP5375R,0.0,0.0,0.0,2018-10-25,B07RY76LGX,0.0,0.0,0.0,2019-05-17
1656,PR410102-RGB-WP-US,B08BZGRJCV,2020-09-11,us,led strip lights,us-th1001-2000,0.0,0.0,0.0,0.7025,...,B07JP5375R,0.0,0.0,0.0,2018-10-25,B07RY76LGX,0.0,0.0,0.0,2019-05-17
1657,PR410102-RGB-WP-US,B08BZGRJCV,2020-09-12,us,led strip lights,us-th1001-2000,0.0,0.0,0.0,0.7600,...,B07JP5375R,0.0,0.0,0.0,2018-10-25,B07RY76LGX,0.0,0.0,0.0,2019-05-17
1658,PR410102-RGB-WP-US,B08BZGRJCV,2020-09-13,us,led strip lights,us-th1001-2000,0.0,0.0,0.0,0.7975,...,B07JP5375R,0.0,0.0,0.0,2018-10-25,B07RY76LGX,0.0,0.0,0.0,2019-05-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54129,100070-DW-US-16,B07DPN99GC,2021-05-08,us,led bulbs,us-hh1-1000,0.0,0.0,0.0,0.0000,...,B019KKFI98,0.0,0.0,0.0,2015-12-19,B07QKN3SFN,0.0,0.0,0.0,2018-11-14
54130,100070-DW-US-16,B07DPN99GC,2021-05-09,us,led bulbs,us-hh1-1000,0.0,0.0,0.0,0.0000,...,B019KKFI98,0.0,0.0,0.0,2015-12-19,B07QKN3SFN,0.0,0.0,0.0,2018-11-14
54131,100070-DW-US-16,B07DPN99GC,2021-05-10,us,led bulbs,us-hh1-1000,0.0,0.0,0.0,0.0000,...,B019KKFI98,0.0,0.0,0.0,2015-12-19,B07QKN3SFN,0.0,0.0,0.0,2018-11-14
54132,100070-DW-US-16,B07DPN99GC,2021-05-11,us,led bulbs,us-hh1-1000,0.0,0.0,0.0,0.0000,...,B019KKFI98,0.0,0.0,0.0,2015-12-19,B07QKN3SFN,0.0,0.0,0.0,2018-11-14


In [8]:
import os
from os.path import join, getsize
path = 'Z:\内部参考资料\预警表（每日文件）'
next(os.walk(path))

('Z:\\内部参考资料\\预警表（每日文件）',
 ['历史记录'],
 ['20210506.zip',
  '20210507.zip',
  '20210510.zip',
  '20210512.zip',
  '20210514.zip',
  '20210517.zip',
  'Thumbs.db',
  '~$1,仓库每日进销存报表-fion.xlsx',
  '~$1,仓库每日进销存报表（无公式版）.xlsx'])

In [4]:
import pandas as pd 
import os
import zipfile
import shutil
from pathlib import Path
from os.path import join, getsize
from datetime import datetime

now_date = datetime.today().strftime('%Y-%m-%d')
path = 'Z:\内部参考资料\预警表（每日文件）'
paths = sorted(Path(path).iterdir(), key=os.path.getmtime)
unzip_dir_path = 'G:\\InternalResource_' + now_date


with zipfile.ZipFile(paths[-1], 'r') as zf:
    # 解压到指定目录,首先创建一个解压目录
    os.mkdir(unzip_dir_path)
    for old_name in zf.namelist():
        file_size = zf.getinfo(old_name).file_size
        new_name = old_name.encode('cp437').decode('gbk')
        new_path = os.path.join(unzip_dir_path, new_name)
        if file_size > 0:
            with open(file=new_path, mode='wb') as f:
                f.write(zf.read(old_name))
        else:
            os.mkdir(new_path)

In [2]:
pis_path = unzip_dir_path + '\\1,仓库每日进销存报表（无公式版）.xlsx'
sto_data = pd.DataFrame()
raw_sto = pd.read_excel(pis_path, sheet_name=None)

for key in list(raw_sto.keys())[1:-1]:
    raw_sto[key].rename(columns=raw_sto[key].iloc[0], inplace=True)
    raw_sto[key] = raw_sto[key].iloc[1:, :]
    raw_sto[key].columns = raw_sto[key].columns.str.replace(
        '\\n', '', regex=True)
    raw_sto[key] = raw_sto[key][['SKUNo.', 'P_SKU', '仓库', '已备未发', '海外库存',
                                    'SZ仓库存数量(除去已经备货)', '工厂已下单的量', '暂收仓待检/不良', '周期', '是否续卖', '等级']]
    raw_sto[key].columns = ['ssku', 'p_sku', 'sto_name', 'stockup', 'oversea_storage',
                            'sz_storage', 'in_factory', 'qc', 'period', 'continue', 'level']
    raw_sto[key] = raw_sto[key][~(raw_sto[key]['sto_name'] == '汇总')]
    raw_sto[key]['storage_unsent'] = raw_sto[key]['stockup'] + \
        raw_sto[key]['oversea_storage']
    raw_sto[key]['storage_sum'] = raw_sto[key]['stockup'] + \
        raw_sto[key]['oversea_storage'] + raw_sto[key]['sz_storage'] + \
        raw_sto[key]['in_factory']+raw_sto[key]['qc']
    sto_data = sto_data.append(raw_sto[key])
    sto_data[['ssku', 'p_sku']] = sto_data[['ssku', 'p_sku']].applymap(lambda x: str(x))

In [34]:
import pandas as pd 
import os
import zipfile
import shutil
from pathlib import Path
from os.path import join, getsize
from datetime import datetime

class DataDeliver:
    path = 'Z:\内部参考资料\预警表（每日文件）'
    paths = sorted(Path(path).iterdir(), key=os.path.getmtime)
    current_date_path = paths[-1]
    file_date = datetime.strptime(str(current_date_path).split('\\')[-1].\
        replace('.zip',''),'%Y%m%d').strftime('%Y-%m-%d')
    print(current_date_path)
    unzip_dir_path = 'G:\\InternalResource_' + file_date
    pis_path = unzip_dir_path + '\\1,仓库每日进销存报表（无公式版）.xlsx'


    def __init__(self):
        pass

    def unzip_data(self, current_date_path=None, unzip_dir_path=None):
        path = 'Z:\内部参考资料\预警表（每日文件）'
        paths = sorted(Path(path).iterdir(), key=os.path.getmtime)
        current_date_path = paths[-1]
        file_date = datetime.strptime(str(current_date_path).split('\\')[-1].\
        replace('.zip',''),'%Y%m%d').strftime('%Y-%m-%d')
        unzip_dir_path = 'G:\\InternalResource_' + file_date

        with zipfile.ZipFile(current_date_path, 'r') as zf:
            os.mkdir(unzip_dir_path)
            for old_name in zf.namelist():
                file_size = zf.getinfo(old_name).file_size
                new_name = old_name.encode('cp437').decode('gbk')
                new_path = os.path.join(unzip_dir_path, new_name)
                if file_size > 0:
                    with open(file=new_path, mode='wb') as f:
                        f.write(zf.read(old_name))
                else:
                    os.mkdir(new_path)

dd = DataDeliver()
dd.unzip_data()

Z:\内部参考资料\预警表（每日文件）\20210521.zip


In [ ]:
import pandas as pd
gn = pd.read_excel('G:\\InternalResource_2021-05-20\\国内库存明细(进销存).xlsx',sheet_name='备货SKU对应出库SKU')
gn = gn[['SKU','P_SKU','倍数']]
gn.columns=['sku','p_sku','ratio']
gn = gn[gn['p_sku'].notna()]
from core.db import db
db.set_db(db_schema='lepro_bi')
with db.con as conn:
    gn.to_sql('sku_psku_ratio',conn,if_exists='replace')
pred_data = pd.read_excel('518预估.xlsx')